# Pre processing UCI-HAR

This notebook will pre processing the dataset UCI-HAR (https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones)

1. Load the uci dataset
2. Remove the overlap from samples
3. Remove the gravity appling a ButterWoeth filter
4. Resample the data from 50Hz to 20Hz
5. Change the time window from 2.56s to 3s
6. Change the accelerometer mensure from g to m/s²

In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

## Filtering the accelerometer signal

In [2]:
from scipy import signal

def filtering(sig):
    
    sos = signal.butter(3, 0.3, 'low', fs=50, output='sos')
    filtered = signal.sosfilt(sos, sig)
    return filtered

def resample(sig, time_window):
    return signal.resample(sig, 20*time_window)

In [3]:
def transformar(X, y, user, sensor, filter_the_signal=True):
    """Essa função é responsável por pegar os dados do UCI-HAR
    (recortados em janelas de 2,56 segundos e com sobreposição de 50%)
    e transformá-los em janelas de 3 segundos sem sobreposição"""
    '''
    '''
    
    # Vamos ver quando ocorre uma nova amostra (ou seja, quando não ocorre sobreposição)
    aux = np.zeros(len(y)-1)
    for i in range(len(y)-1):
        aux[i] = np.all(X[i,64:]!=X[i+1,:64])
    ind = np.nonzero(aux)[0]+1
    ind = np.append([0],ind)
    ind = np.append(ind,[len(y)])
    
    # Agora vamos construir as capturas originais (sem sobreposição)
    sigs = []
    yy = []
    uuser = []
    for i in range(len(ind)-1):
        sig = X[ind[i]:ind[i+1],64:].reshape((ind[i+1]-ind[i])*64)
        sig = np.append(X[ind[i],:64],sig)
        
        if sensor < 3:
            if filter_the_signal:
                sig_filtered = sig - filtering(sig) # Removing the gravity
            else:
                sig_filtered = sig
            time_window = len(sig_filtered) // 50 # The time window is the time of the total window
            sig_resampled = resample(sig_filtered, time_window) # Resampling the signal

            sigs.append(sig_resampled)
            yy.append(y[ind[i]])
            uuser.append(user[ind[i]])
        else:
            time_window = len(sig) // 50 # The time window is the time of the total window
            sig_resampled = resample(sig, time_window) # Resampling the signal

            sigs.append(sig_resampled)
            yy.append(y[ind[i]])
            uuser.append(user[ind[i]])
    # Vamos construir as janelas de 3 segundos (lembrando que o UCI-HAR está amostrado a uma taxa de 20 Hz)
    new_time_window = 3
    sample_rate = 20
    points_per_new_window = new_time_window*sample_rate
    
    aux = np.zeros(len(sigs))
    for i in range(len(sigs)):
        aux[i] = len(sigs[i])
    Na = np.sum(aux//points_per_new_window, dtype=np.int64)
    mat = np.zeros((Na,points_per_new_window))
    yyy = np.zeros(Na)
    uuuser = np.zeros(Na)
    k = 0
    for i in range(len(sigs)):
        for j in range(len(sigs[i])//points_per_new_window):
            mat[k,:] = sigs[i][j*points_per_new_window:(j+1)*points_per_new_window]
            yyy[k] = yy[i]
            uuuser[k] = uuser[i]
            k+=1
    return mat, yyy, uuuser;

## Pre processing the train data

In [4]:
# Vamos abrir os dados de treino do UCI-HAR
pasta = "../../../../../Downloads/UCI HAR Dataset/UCI HAR Dataset/train/"
# dados = [None]*6
sensor = [
    'Inertial Signals/body_acc_x_train.txt', 
    'Inertial Signals/body_acc_y_train.txt', 
    'Inertial Signals/body_acc_z_train.txt', 
    'Inertial Signals/body_gyro_x_train.txt', 
    'Inertial Signals/body_gyro_y_train.txt', 
    'Inertial Signals/body_gyro_z_train.txt'
]
dados = [np.loadtxt(pasta+sensor[i]) for i in range(6)]
                    
# dados[0] = np.loadtxt(pasta+'Inertial Signals/total_acc_x_train.txt')
# dados[1] = np.loadtxt(pasta+'Inertial Signals/total_acc_y_train.txt')
# dados[2] = np.loadtxt(pasta+'Inertial Signals/total_acc_z_train.txt')
# dados[3] = np.loadtxt(pasta+'Inertial Signals/body_gyro_x_train.txt')
# dados[4] = np.loadtxt(pasta+'Inertial Signals/body_gyro_y_train.txt')
# dados[5] = np.loadtxt(pasta+'Inertial Signals/body_gyro_z_train.txt')
                    
y = np.loadtxt(pasta+'y_train.txt')
user = np.loadtxt(pasta+'subject_train.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 1226
WALKING_UPSTAIRS: 1073
WALKING_DOWNSTAIRS: 986
SITTING: 1286
STANDING: 1374
LAYING: 1407


In [5]:
new_time_window = 3
sample_rate = 20
points_per_new_window = new_time_window*sample_rate

colunas = ['accel-x-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['accel-y-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['accel-z-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-x-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-y-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['gyro-z-' + str(x) for x in np.arange(points_per_new_window).tolist()]
colunas += ['activity code']
colunas += ['user']

In [6]:
new_time_window = 3
sample_rate = 20
points_per_new_window = new_time_window*sample_rate

train = []
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user, i, filter_the_signal=False)
    if type(train) == list:
        train = mat
    else:
        train = np.concatenate([train, mat], axis=1)
    print(train.shape)
train = np.column_stack((train,yyy.astype(int),uuuser.astype(int)))

(3049, 60)
(3049, 120)
(3049, 180)
(3049, 240)
(3049, 300)
(3049, 360)


In [7]:
df_train = pd.DataFrame(data=train)

In [8]:
df_train.columns = colunas
df_train = df_train.astype({"activity code": int, "user": int})

# Removing the activity laying
df_train = df_train[df_train['activity code'].isin([1, 2, 3, 4, 5])]
df_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,0.002387,0.009461,0.005275,0.005407,0.005928,0.006327,0.005545,0.006627,0.006874,0.006595,...,0.001605,-0.000460,-0.001029,-0.000238,0.002616,0.006758,0.009952,-0.005709,5,1
1,-0.000674,-0.000546,0.002028,-0.000696,-0.000987,0.000994,0.004223,-0.000982,0.000872,-0.000745,...,-0.001788,-0.005908,-0.010234,-0.011284,-0.009106,-0.001114,-0.001296,-0.002068,5,1
2,-0.000667,-0.001996,-0.001186,0.001180,0.000665,-0.001952,-0.000154,-0.001065,0.000771,-0.000316,...,0.000421,0.004503,0.002473,0.001083,0.007256,0.021702,0.007641,0.009008,5,1
3,-0.003861,0.001007,-0.000125,0.000563,0.004378,-0.002226,-0.000344,-0.001331,0.001893,0.001811,...,0.005918,-0.003793,-0.002657,0.001718,-0.007073,-0.012203,-0.009020,-0.008270,5,1
4,0.001380,0.001724,-0.002955,0.002529,-0.000079,0.000654,-0.002053,-0.001705,0.004524,0.000213,...,0.000288,-0.001961,0.001915,0.002949,0.007039,-0.004036,-0.003410,-0.004388,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,-0.041896,0.052157,0.041933,0.132608,0.403926,0.584520,0.314604,0.024553,-0.053856,-0.144348,...,-0.396459,-0.545852,-0.702665,-0.833772,-0.654115,-0.355501,-0.202928,-0.085239,2,30
3045,0.005111,0.189568,0.440365,0.472448,0.247937,-0.024209,-0.138177,-0.148464,-0.374727,-0.347748,...,-0.445656,-0.579521,-0.846560,-0.700226,-0.574337,-0.199706,0.069693,0.210308,2,30
3046,0.264501,0.521548,0.476120,0.183919,-0.034222,-0.064747,-0.176820,-0.332799,-0.341727,-0.357113,...,0.035472,0.292552,0.203059,-0.175440,0.281563,0.134684,0.217240,0.443779,2,30
3047,-0.247902,-0.389041,-0.203186,-0.252128,-0.237445,-0.134098,0.074885,0.285351,0.717716,0.160373,...,-0.232415,0.024815,0.065954,0.068590,0.115724,0.091088,0.309496,0.256306,2,30


In [9]:
print('Número de amostras por classe na formatação final:')
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 506
WALKING_UPSTAIRS: 439
WALKING_DOWNSTAIRS: 395
SITTING: 544
STANDING: 575
LAYING: 590


In [10]:
df_train.iloc[:,:181]

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,accel-z-51,accel-z-52,accel-z-53,accel-z-54,accel-z-55,accel-z-56,accel-z-57,accel-z-58,accel-z-59,gyro-x-0
0,0.002387,0.009461,0.005275,0.005407,0.005928,0.006327,0.005545,0.006627,0.006874,0.006595,...,-0.004354,0.002257,-0.012109,-0.013924,-0.004051,-0.005563,0.000682,0.000204,0.001344,0.024057
1,-0.000674,-0.000546,0.002028,-0.000696,-0.000987,0.000994,0.004223,-0.000982,0.000872,-0.000745,...,-0.008617,-0.003289,-0.005607,-0.006317,-0.002590,0.001681,-0.002794,-0.009199,-0.005793,0.012282
2,-0.000667,-0.001996,-0.001186,0.001180,0.000665,-0.001952,-0.000154,-0.001065,0.000771,-0.000316,...,0.001418,-0.000769,0.002482,0.002587,0.001348,-0.000136,0.003699,-0.007203,0.011626,-0.007669
3,-0.003861,0.001007,-0.000125,0.000563,0.004378,-0.002226,-0.000344,-0.001331,0.001893,0.001811,...,0.001057,-0.004921,-0.002995,0.003872,0.006467,0.005734,-0.000312,0.005372,-0.002784,-0.003489
4,0.001380,0.001724,-0.002955,0.002529,-0.000079,0.000654,-0.002053,-0.001705,0.004524,0.000213,...,0.005009,0.000475,0.006699,0.004116,0.002461,0.009960,-0.004025,-0.001737,0.009420,0.009188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,-0.041896,0.052157,0.041933,0.132608,0.403926,0.584520,0.314604,0.024553,-0.053856,-0.144348,...,0.158199,-0.001713,-0.036485,-0.102678,-0.048432,-0.147519,-0.182713,-0.187170,-0.162145,0.322930
3045,0.005111,0.189568,0.440365,0.472448,0.247937,-0.024209,-0.138177,-0.148464,-0.374727,-0.347748,...,0.002842,-0.086868,-0.089941,-0.073893,-0.146606,-0.175663,-0.145740,-0.134318,-0.175687,-0.493072
3046,0.264501,0.521548,0.476120,0.183919,-0.034222,-0.064747,-0.176820,-0.332799,-0.341727,-0.357113,...,-0.113406,-0.129162,-0.111990,-0.213836,-0.396565,-0.240834,-0.220076,-0.165139,-0.133147,-0.517789
3047,-0.247902,-0.389041,-0.203186,-0.252128,-0.237445,-0.134098,0.074885,0.285351,0.717716,0.160373,...,-0.156355,-0.075124,-0.039430,-0.086528,-0.059930,0.031654,0.140635,0.201567,0.328916,0.519843


In [11]:
# Mudando a unidade de medida do acelerômetro para m/s² (Basta multiplicar por g = 9.81)
# g = 9.81
g = 9.81
df_train.iloc[:,:180] = df_train.iloc[:,:180]*g
df_train

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,0.023421,0.092810,0.051748,0.053044,0.058151,0.062064,0.054399,0.065013,0.067435,0.064698,...,0.001605,-0.000460,-0.001029,-0.000238,0.002616,0.006758,0.009952,-0.005709,5,1
1,-0.006615,-0.005355,0.019899,-0.006828,-0.009687,0.009754,0.041427,-0.009636,0.008558,-0.007307,...,-0.001788,-0.005908,-0.010234,-0.011284,-0.009106,-0.001114,-0.001296,-0.002068,5,1
2,-0.006539,-0.019581,-0.011634,0.011580,0.006525,-0.019151,-0.001512,-0.010446,0.007561,-0.003103,...,0.000421,0.004503,0.002473,0.001083,0.007256,0.021702,0.007641,0.009008,5,1
3,-0.037877,0.009880,-0.001231,0.005519,0.042951,-0.021841,-0.003378,-0.013059,0.018568,0.017769,...,0.005918,-0.003793,-0.002657,0.001718,-0.007073,-0.012203,-0.009020,-0.008270,5,1
4,0.013536,0.016908,-0.028985,0.024807,-0.000772,0.006416,-0.020137,-0.016722,0.044381,0.002091,...,0.000288,-0.001961,0.001915,0.002949,0.007039,-0.004036,-0.003410,-0.004388,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,-0.411000,0.511663,0.411359,1.300886,3.962514,5.734145,3.086267,0.240866,-0.528328,-1.416055,...,-0.396459,-0.545852,-0.702665,-0.833772,-0.654115,-0.355501,-0.202928,-0.085239,2,30
3045,0.050143,1.859659,4.319982,4.634719,2.432258,-0.237494,-1.355517,-1.456428,-3.676076,-3.411408,...,-0.445656,-0.579521,-0.846560,-0.700226,-0.574337,-0.199706,0.069693,0.210308,2,30
3046,2.594754,5.116385,4.670734,1.804248,-0.335719,-0.635171,-1.734604,-3.264757,-3.352341,-3.503276,...,0.035472,0.292552,0.203059,-0.175440,0.281563,0.134684,0.217240,0.443779,2,30
3047,-2.431914,-3.816490,-1.993255,-2.473374,-2.329336,-1.315504,0.734617,2.799288,7.040791,1.573255,...,-0.232415,0.024815,0.065954,0.068590,0.115724,0.091088,0.309496,0.256306,2,30


## Pre processing the test data

In [12]:
# Agora vamos abrir os dados de treino do UCI-HAR
pasta = "../../../../../Downloads/UCI HAR Dataset/UCI HAR Dataset/test/"
dados = [None]*6
sensor = [
    'Inertial Signals/body_acc_x_test.txt', 
    'Inertial Signals/body_acc_y_test.txt', 
    'Inertial Signals/body_acc_z_test.txt', 
    'Inertial Signals/body_gyro_x_test.txt', 
    'Inertial Signals/body_gyro_y_test.txt', 
    'Inertial Signals/body_gyro_z_test.txt'
]
dados = [np.loadtxt(pasta+sensor[i]) for i in range(6)]
                    
y = np.loadtxt(pasta+'y_test.txt')
user = np.loadtxt(pasta+'subject_test.txt')
labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING'
          , 'LAYING']
print('Número de amostras por classe \npara janelas de 2,56 segundos e sobreposição de 50%')
for i in np.unique(y):
    print(labels[i.astype(int)-1]+': '+str(np.sum(y==i)))

Número de amostras por classe 
para janelas de 2,56 segundos e sobreposição de 50%
WALKING: 496
WALKING_UPSTAIRS: 471
WALKING_DOWNSTAIRS: 420
SITTING: 491
STANDING: 532
LAYING: 537


In [13]:
new_time_window = 3
sample_rate = 20
points_per_new_window = new_time_window*sample_rate

test = []
for i in range(6):
    mat, yyy, uuuser = transformar(dados[i], y, user, i, filter_the_signal=False)
    if type(test) == list:
        test = mat
    else:
        test = np.concatenate([test, mat], axis=1)
    print(test.shape)
test = np.column_stack((test,yyy.astype(int),uuuser.astype(int)))

(1224, 60)
(1224, 120)
(1224, 180)
(1224, 240)
(1224, 300)
(1224, 360)


In [14]:
df_test = pd.DataFrame(data=test)

In [15]:
df_test.columns = colunas
df_test = df_test.astype({"activity code": int, "user": int})

# Removing the activity laying
df_test = df_test[df_test['activity code'].isin([1, 2, 3, 4, 5])]
df_test

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,-0.003013,0.026373,-0.034742,-0.085930,-0.022595,-0.015299,-0.014436,-0.020153,-0.025227,-0.004829,...,0.020261,0.019462,0.031394,0.031187,0.028060,0.034830,0.026174,0.013201,5,2
1,-0.000351,-0.000118,0.003072,0.004772,-0.001883,-0.002401,-0.003273,0.004967,0.001555,-0.002361,...,-0.010148,-0.007775,-0.005448,-0.002262,-0.005768,-0.007097,-0.003873,-0.005801,5,2
2,0.001581,0.001253,-0.001897,0.000558,-0.003485,-0.003852,0.002102,0.007151,0.003556,-0.000824,...,-0.031461,-0.035463,-0.008491,-0.002806,0.007795,0.011203,0.003488,-0.004301,5,2
3,-0.000112,0.001339,0.000591,0.002149,0.001728,0.004761,0.004666,0.001528,0.002537,-0.000683,...,-0.010083,-0.005960,-0.006940,-0.009337,-0.009792,-0.001859,-0.004210,-0.000520,5,2
4,-0.000428,-0.001493,-0.005389,-0.001481,-0.002452,-0.001556,-0.001542,0.002294,0.000597,-0.002573,...,0.003836,0.001970,0.003919,0.005948,-0.002991,0.006746,0.007876,0.006598,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,-0.265079,-0.366498,-0.212961,-0.090736,0.019230,0.259044,0.618721,0.426178,0.058716,-0.050731,...,0.117826,-0.216337,-0.498443,-0.221196,0.040862,0.122537,0.130663,0.001037,3,24
1220,-0.018231,0.275457,0.287516,0.222868,0.390017,0.008049,-0.174194,-0.062345,-0.065193,-0.162506,...,-0.128649,0.097912,0.551742,0.356796,-0.128970,-0.070716,0.436967,-0.035641,2,24
1221,0.072077,-0.038182,-0.159871,-0.024240,-0.174263,-0.236928,-0.202852,-0.121782,-0.029389,0.067459,...,-0.147423,-0.101344,-0.070548,0.196283,0.036365,-0.383925,-0.499610,-0.496780,2,24
1222,-0.001646,-0.146423,-0.321656,-0.254214,-0.090506,-0.092728,0.070351,0.132218,0.169886,0.414673,...,-0.345504,-0.396574,-0.585145,-0.923041,-0.865991,-0.622471,-0.736700,-0.851594,2,24


In [16]:
print('Número de amostras por classe na formatação final:')
for i in np.unique(yyy):
    print(labels[i.astype(int)-1]+': '+str(np.sum(yyy==i)))

Número de amostras por classe na formatação final:
WALKING: 204
WALKING_UPSTAIRS: 189
WALKING_DOWNSTAIRS: 173
SITTING: 204
STANDING: 227
LAYING: 227


In [17]:
# Mudando a unidade de medida do acelerômetro para m/s² (Basta multiplicar por g = 9.81)
# g = 9.81
g = 9.81
df_test.iloc[:,:180] = df_test.iloc[:,:180]*g
df_test

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-52,gyro-z-53,gyro-z-54,gyro-z-55,gyro-z-56,gyro-z-57,gyro-z-58,gyro-z-59,activity code,user
0,-0.029561,0.258717,-0.340819,-0.842972,-0.221659,-0.150078,-0.141619,-0.197705,-0.247472,-0.047373,...,0.020261,0.019462,0.031394,0.031187,0.028060,0.034830,0.026174,0.013201,5,2
1,-0.003447,-0.001159,0.030140,0.046814,-0.018468,-0.023551,-0.032113,0.048727,0.015255,-0.023159,...,-0.010148,-0.007775,-0.005448,-0.002262,-0.005768,-0.007097,-0.003873,-0.005801,5,2
2,0.015510,0.012292,-0.018606,0.005475,-0.034190,-0.037785,0.020620,0.070149,0.034882,-0.008088,...,-0.031461,-0.035463,-0.008491,-0.002806,0.007795,0.011203,0.003488,-0.004301,5,2
3,-0.001102,0.013132,0.005802,0.021082,0.016951,0.046706,0.045773,0.014992,0.024891,-0.006696,...,-0.010083,-0.005960,-0.006940,-0.009337,-0.009792,-0.001859,-0.004210,-0.000520,5,2
4,-0.004203,-0.014642,-0.052864,-0.014527,-0.024052,-0.015269,-0.015128,0.022507,0.005856,-0.025241,...,0.003836,0.001970,0.003919,0.005948,-0.002991,0.006746,0.007876,0.006598,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,-2.600420,-3.595345,-2.089147,-0.890123,0.188650,2.541224,6.069649,4.180803,0.576008,-0.497670,...,0.117826,-0.216337,-0.498443,-0.221196,0.040862,0.122537,0.130663,0.001037,3,24
1220,-0.178847,2.702237,2.820533,2.186331,3.826064,0.078957,-1.708842,-0.611605,-0.639540,-1.594183,...,-0.128649,0.097912,0.551742,0.356796,-0.128970,-0.070716,0.436967,-0.035641,2,24
1221,0.707076,-0.374569,-1.568337,-0.237794,-1.709518,-2.324260,-1.989979,-1.194680,-0.288309,0.661777,...,-0.147423,-0.101344,-0.070548,0.196283,0.036365,-0.383925,-0.499610,-0.496780,2,24
1222,-0.016146,-1.436414,-3.155442,-2.493840,-0.887867,-0.909666,0.690148,1.297057,1.666583,4.067945,...,-0.345504,-0.396574,-0.585145,-0.923041,-0.865991,-0.622471,-0.736700,-0.851594,2,24


In [18]:
df_train.to_csv('../../data_2/views/UCI-HAR/unbalanced_view_without_gravity_acc_9.81_train_test-resampled_20hz-v1/train.csv',index=False)
df_test.to_csv('../../data_2/views/UCI-HAR/unbalanced_view_without_gravity_acc_9.81_train_test-resampled_20hz-v1/test.csv',index=False)